## Lotte Bottema 11269642
## Kaj Meijer 10509534

**Table of contents**

* [Task](#task)
* [Naive Bayes](#NB)
    * [Feature function](#ff)
    * [MLE](#MLE)
    * [Evaluation](#eval)

    
**Table of Exercises**


* [Exercise 5-1](#ex5-1) (-/1)
* [Exercise 5-2](#ex5-2) (-/2)
* [Exercise 5-3](#ex5-3) (-/3)
* [Exercise 5-4](#ex5-4) (-/2)
* [Exercise 5-5](#ex5-5) (-/4)



**General notes**

* In this notebook you are expected to use $\LaTeX$. 
* Use python3.
* Use NLTK to read annotated data.
* **Document your code**: TAs are more likely to understand the steps if you document them. If you don't, it's also difficult to give you partial points for exercises that are not completely correct.

After completing this lab you should be able to 

* develop Naive Bayes text classifiers
* estimate parameters via MLE
* predict and evaluate models using precision/recall

# <a name="task"> Task

We will be looking into binary sentiment analysis where we have to decide whether a document $x$ (a list of tokens) is positive (class $y=1$) or negative (class $y=0$) towards a subject.

The dataset we will use comes from NLTK [nltk.corpus.sentence_polarity](http://www.nltk.org/howto/corpus.html):

In [1]:
from nltk.corpus import sentence_polarity

This dataset contains 5331 positive and 5331 negative sentences, which you can obtain as shown below:

In [2]:
pos_sents = sentence_polarity.sents(categories='pos')
neg_sents = sentence_polarity.sents(categories='neg')
print(len(pos_sents), 'positive sentences such as:\n', ' '.join(pos_sents[0]))
print(len(neg_sents), 'negative sentences such as:\n', ' '.join(neg_sents[0]))

5331 positive sentences such as:
 the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
5331 negative sentences such as:
 simplistic , silly and tedious .


We will use the first 4000 sentences from each class for training, the next 331 for development, and the last 1000 for test:

In [3]:
training_pos = pos_sents[:4000]
training_neg = neg_sents[:4000]
dev_pos = pos_sents[4000:4331]
dev_neg = neg_sents[4000:4331]
test_pos = pos_sents[4331:]
test_neg = neg_sents[4331:]
print('Training: %d pos and %d neg' % (len(training_pos), len(training_neg)))
print('Development: %d pos and %d neg' % (len(dev_pos), len(dev_neg)))
print('Test: %d pos and %d neg' % (len(test_pos), len(test_neg)))

Training: 4000 pos and 4000 neg
Development: 331 pos and 331 neg
Test: 1000 pos and 1000 neg


In [4]:
training_docs = training_pos + training_neg
dev_docs = dev_pos + dev_neg
test_docs = test_pos + test_neg

# <a name="NB"> Naive Bayes


Feature-rich models are used to model the distribution $P_{Y|X}(y|x)$ of a target variable $y$ conditioned on some high-dimensional data $x$.

One way of doing it is to summarise aspects of $x$ that are relevant to the problem by means of a feature function which returns a vector in some subset of $\mathbb R^D$. For example, this feature function may retain sentiment words in $x$ or some other important aspects of the input. Then instead of modelling $P_{Y|X}(y|x)$ we can, for example, model $P_{Y|F_1^n}(y|f_1^n)$ where we condition on a collection of $n$ features instead.

Conditioning on features of the input, rather than the input directly, does not address the problem on its own, that is, the conditioning context remains high-dimensional. But here is where we can use probability calculus and independence assumptions to make our task simpler.

We can use Bayes rule to invert this conditional:

\begin{align}
(1) \quad P_{Y|F_1^n}(y|f_1^n) = \frac{P_Y(y)P_{F_1^n|Y}(f_1^n|y)}{P_{F_1^n}(f_1^n)}
\end{align}

Now note that the numerator has a conditional where the high-dimensional feature representation of the input is modelled from the target class. That is a problem we can address by making conditional independence assumptions. In particular, by making $F_i$ independent on every other $F_j$ with $i \neq j$ given the target label $y$ we can simplify the problem a lot. We denote this by $F_i \perp F_j \mid y$ for $i\neq j$. Equation (2) shows the resulting model:

\begin{align}
(2) \quad P_{Y|F_1^n}(y|f_1^n) \overset{\text{ind}}{=} \frac{P_Y(y)\prod_{i=1}^n P_{F|Y}(f_i|y)}{P_{F_1^n}(f_1^n)}
\end{align}


Note that we have to model fairly small cpds now:
* a *prior* distribution over classes $P_Y(y)$ 
* a set of cpds $P_{F|Y}$, one per class, over the possible features (these distributions are also called likelihoods, but do not confuse it with the *likelihood function* which is a function of parameters of a statistical model for fixed data)
* the denominator can be inferred by marginalisation, see Equation (3)

\begin{align}
(2) \quad P_{F_1^n}(f_1^n) = \sum_{y \in \mathcal Y} P_{YF_1^n}(y, f_1^n) = \sum_{y \in \mathcal Y} P_{Y}(y)P_{F_1^n|Y}(f_1^n|y) = \sum_{y \in \mathcal Y} P_{Y}(y) \prod_{i=1}^n P_{F|Y}(f_i|y)
\end{align}



## <a name="ff"> Feature function

An important part of a feature-rich model such as Naive Bayes (NB) classifiers is the *feature function*. Here we will develop one. 

In NB classification, features are themselves random variables defined over a certain set $\mathcal F$. We need to first determine this set. In this notebook we will focus on *unigram features*, that is, features defined at the token level.

We will take every token that occurs more than a pre-specified number of times as a potential feature.

Here is an example of how you can do this:

In [5]:
from collections import Counter


def make_unigram_feature_set(documents, min_freq=1, mark_negation=False):
    """
    This function goes through a corpus and retains all candidate unigram features
     making a feature set. 
    
    :param documents: all documents, each a list of words
    :param min_freq: minimum frequency of a token for it to be part of the feature set
    :param mark_negation: **IGNORE THIS FOR NOW**
    :returns: unigram feature set
    """
    counter = Counter()
    for doc in documents:
        counter.update(doc)
    features = []
    for f, n in counter.most_common():
        if n >= min_freq:
            features.append(f)
        else:
            break
    return frozenset(features)

<a name="ex5-1" style="color:red">**Exercise 5-1**</a> **[1 points]** Modify `make_unigram_feature_set` to optionally pre-process documents by marking words in the scope of a negation with the suffix `_NEG`. For example,  `I am not sure I like the acting` becomes `I am not sure_NEG I_NEG like_NEG the_NEG acting_NEG`. You can use NLTK support for that, see for example, `nltk.sentiment.util.mark_negation`.

In [6]:
# You can check python documentation by using the following syntax
from nltk.sentiment import util
# util.mark_negation?

In [7]:
from collections import Counter

def make_unigram_feature_set(documents, min_freq=1, mark_negation=False):
    """
    This function goes through a corpus and retains all candidate unigram features
     making a feature set. Optionally, it can also preprocess the corpus annotating
     with _NEG words that are in the scope of a negation (using NLTK helper functions).
    
    :param documents: all documents, each a list of words
    :param min_freq: minimum frequency of a token for it to be part of the feature set
    :param mark_negation: whether to preprocess the document using NLTK's nltk.sentiment.util.mark_negation
        see the documentation `nltk.sentiment.util.mark_negation?`
    :returns: unigram feature set
    """
    # create a counter
    counter = Counter()
    
    # loop over the documents
    for doc in documents:
        
        # mark negations
        if mark_negation:
            
            # update the counter with the negations
            counter.update(util.mark_negation(doc))
            
        # do not mark negations
        else:
            
            # update the counter without negations
            counter.update(doc)
            
    # features list
    features = []
    
    # get the feature and occurences from the counter
    # in order from highest to lowest n
    for f, n in counter.most_common():
        
        # only include features that are frequent enough
        if n >= min_freq:
            
            # add the feature to the list 
            features.append(f)
        
        # lower, so everything that follows will be lower
        else:
            
            # break the loop
            break
    
    # freeze the set of features
    return frozenset(features)

In [8]:
# This is just some helper code for better visualization of examples
def inspect_set(input_set, k=5, neg=False):
    """
    Helper function to inspect a few elements in a set of features
         with _NEG words that are in the scope of a negation (using NLTK helper functions).
    :param documents: all documents, each a list of words
    :param input_set: a set of features
    :param k: how many elements to select
    :param neg: return `*_NEG` features only
    :returns: up to k elements 
    """
    selected = set()
    for w in input_set:
        if len(selected) < k:            
            if not neg:
                selected.add(w)
            elif '_NEG' in w:
                selected.add(w)
        else:
            break
    return selected

Here are some of the features (without marking negation):

```python
>>> unigram_features = make_unigram_feature_set(training_docs+dev_docs, min_freq=2)
>>> print(len(unigram_features), 'features such as:\n', inspect_set(unigram_features))
```
```
9059 features such as:
 {'white', 'fearless', 'ear', 'tempting', 'meat'}
```

In [9]:
unigram_features = make_unigram_feature_set(training_docs+dev_docs, min_freq=2)
print(len(unigram_features), 'features such as:\n', inspect_set(unigram_features))

9059 features such as:
 {'barbed', 'explore', "'", 'partnerships', 'scenery'}


Here are some of the features with pre-processing negation scope.

```python
>>> unigram_features_with_negation = make_unigram_feature_set(training_docs+dev_docs, min_freq=2, mark_negation=True)
>>> print(len(unigram_features_with_negation), 'features such as:\n', 
      inspect_set(unigram_features_with_negation), 
      '\nand:\n', inspect_set(unigram_features_with_negation, neg=True))
```

```
10143 features such as:
 {'white', 'message_NEG', 'fearless', 'ear', 'tempting'} 
and:
 {'ticket_NEG', 'message_NEG', 'street_NEG', 'determined_NEG', 'stereotype_NEG'}     
```

In [10]:
unigram_features_with_negation = make_unigram_feature_set(training_docs+dev_docs, min_freq=2, mark_negation=True)
print(len(unigram_features_with_negation), 'features such as:\n', 
      inspect_set(unigram_features_with_negation), 
      '\nand:\n', inspect_set(unigram_features_with_negation, neg=True))

10143 features such as:
 {'word_NEG', 'explore', "'", 'partnerships', 'scenery'} 
and:
 {'word_NEG', 'things_NEG', 'relentlessly_NEG', 'test_NEG', 'escapism_NEG'}


Now that we know which features will form the basis of our classifier, we need to implement a feature function. Here we call it a feature map (as we will be using a python dictionary).

In NB classification only the features that occur in an input matter for classification, thus we use a dictionary that maps features to their values if they occur and not otherwise.

This function should take a document $x$ and produce a dict where `f` (a feature) is either 1 (for binary features) or a count (for count features). For the purpose of readability we like to represent features with strings, for example:

* `contains(like) = 1` means that the input contains the word `like`
* `count(like) = 3` means that the input contains 3 occurrences of the word `like`
* `EMPTY() = 1` means that the input contains no known feature

<a name="ex5-2" style="color:red">**Exercise 5-2**</a> **[2 points]** Read the documentation below and implement the feature function described.

In [11]:
from collections import defaultdict

def make_feature_map(document, feature_set, 
                     binary=True, 
                     mark_negation=False):
    """
    This function takes a document, possibly pre-processes it by marking words in the scope of negation, 
     and constructs a dict indicating which features in `feature_set` fire. Features may be binary, 
     flagging occurrence, or integer, indicating the number of occurrences.
     If no feature can be extracted, a special feature is fired, namely 'EMPTY()'.
     
    :param document: a list of words
    :param feature_set: set of features we are looking for
    :param binary: whether we are indicating presence or counting features in feature_set
    :param mark_negation: whether we should apply NLTK's mark_negation to document before applying the feature function
    :returns: dict with entries 'contains(f)=1/0' for binary features or 'count(f)=n' for count features
    """
    
    # create the map
    feature_map = defaultdict(float)
    
    # checks if the negations should be marked
    if mark_negation:
        
        # mark the negations in the document
        document = util.mark_negation(document)
    
    # loop over the words in the document
    for word in document:
        
        # check if the word is in the feature set
        if word in feature_set:
            
            # make a binary feature map
            if binary:
                
                # add the word to the map
                feature_map['contains(' + word + ')'] = 1.0
                
            # make a counting feature map
            else:
    
                # add a count of the word to the map
                feature_map['count(' + word + ')'] += 1.0
    
    # check if the map is empty
    if not feature_map:
        
        # mark it as empty
        feature_map['EMPTY()'] = 1.0
       
    # return the map
    return feature_map
        

Here are some outputs for you to check your implementation:

```python
>>> make_feature_map(pos_sents[7], unigram_features_with_negation, 
                 binary=True, mark_negation=True)
```
```
defaultdict(float,
            {'contains(.)': 1.0,
             'contains(ever_NEG)': 1.0,
             'contains(good_NEG)': 1.0,
             'contains(has_NEG)': 1.0,
             'contains(hell_NEG)': 1.0,
             'contains(is_NEG)': 1.0,
             'contains(literally_NEG)': 1.0,
             'contains(made_NEG)': 1.0,
             'contains(more_NEG)': 1.0,
             'contains(no)': 1.0,
             'contains(perhaps)': 1.0,
             'contains(picture_NEG)': 1.0,
             'contains(road_NEG)': 1.0,
             'contains(that_NEG)': 1.0,
             'contains(the_NEG)': 1.0,
             'contains(to_NEG)': 1.0,
             'contains(with_NEG)': 1.0})
```
```python
>>> make_feature_map(['AKSJDHAU'], unigram_features_with_negation, 
                 binary=True, mark_negation=True)
```
```
defaultdict(float, {'EMPTY()': 1.0})
```

In [12]:
make_feature_map(pos_sents[7], unigram_features_with_negation, binary=True, mark_negation=True)

defaultdict(float,
            {'contains(.)': 1.0,
             'contains(ever_NEG)': 1.0,
             'contains(good_NEG)': 1.0,
             'contains(has_NEG)': 1.0,
             'contains(hell_NEG)': 1.0,
             'contains(is_NEG)': 1.0,
             'contains(literally_NEG)': 1.0,
             'contains(made_NEG)': 1.0,
             'contains(more_NEG)': 1.0,
             'contains(no)': 1.0,
             'contains(perhaps)': 1.0,
             'contains(picture_NEG)': 1.0,
             'contains(road_NEG)': 1.0,
             'contains(that_NEG)': 1.0,
             'contains(the_NEG)': 1.0,
             'contains(to_NEG)': 1.0,
             'contains(with_NEG)': 1.0})

In [13]:
make_feature_map(['AKSJDHAU'], unigram_features_with_negation, binary=True, mark_negation=True)

defaultdict(float, {'EMPTY()': 1.0})

In [14]:
make_feature_map([word for sentence in pos_sents[:10] for word in sentence], 
                 unigram_features_with_negation, binary=False, mark_negation=True)

defaultdict(float,
            {'count(")': 4.0,
             'count(,)': 3.0,
             'count(--)': 1.0,
             'count(.)': 13.0,
             'count(21st)': 1.0,
             'count(;)': 1.0,
             'count(a)': 5.0,
             'count(absolute)': 1.0,
             'count(adequately)': 1.0,
             'count(all)': 1.0,
             'count(an)': 1.0,
             'count(and)': 3.0,
             'count(arnold)': 1.0,
             'count(as)': 1.0,
             'count(asian)': 1.0,
             'count(at)': 1.0,
             'count(be)': 1.0,
             'count(biopic)': 1.0,
             'count(but)': 2.0,
             'count(cannot)': 1.0,
             'count(care)': 1.0,
             'count(cat)': 1.0,
             'count(cinema)': 1.0,
             'count(clever)': 1.0,
             'count(co-writer/director)': 1.0,
             'count(column)': 1.0,
             'count(combination)': 1.0,
             'count(comics)': 1.0,
             'count(conan)': 1.0,
     

## <a name="MLE"> MLE

Now you will estimate the cpds involved in NB classification. We use a balanced dataset over two classes (positive and negative), so there's no need to compute $P_Y(y)$, it would simply be $0.5$ per class.

You should simply implement cpds for $P_{F|Y}$, that is, exactly $2$ cpds via MLE and you should use Laplace-$\alpha$ smoothing.

<a name="ex5-3" style="color:red">**Exercise 5-3**</a> **[3 points]** Check the documentation below and complete the code for the NB classifier. You will need to implement

* estimation of cpds $P_{F|Y}$ with Laplace smoothing  (1 point) 
* the `classify` method (2 points)

In [15]:
import numpy as np

def make_cpd(raw_counts, alpha, v):
    """
    This converts a dictionary of raw counts into a cpd.

    :param raw_counts: dict where a key is a feature and a value is its counts (without pseudo counts)
        this should already include the 'EMPTY()' feature
    :param alpha: how many pseudo counts should we add per observation
    :param v: the size of the feature set (already including the 'EMPTY()' feature)
    :returns: a cpd as a dict where a key is a feature and a value is its smoothed probability
    """
    
    # create the cpd dict
    cpd = defaultdict(float)      
    
    # get the total number of words
    total = sum(raw_counts.values())
    
    # loop over the counted words
    for word, count in raw_counts.items():

        # get the smoothed probability ( count + alpha / N + v * alpha )
        cpd[word] = (count + alpha) / (total + ((v-1)*alpha))
   
    # return the cpd
    return cpd


class NaiveBayesClassifier:
    
    def __init__(self, training_pos, training_neg, binary, mark_negation, alpha=0.1, min_freq=2):
        """
        :param training_pos: positive documents
            a document is a list of tokens
        :param training_neg: negative documents
            a document is a list of tokens
        :param binary: whether we are using binary or count features
        :param mark_negation: whether we are pre-processing words in negation scope
        :param alpha: Laplace smooth pseudo count
        :param min_freq: minimum frequency of a token for it to be considered a feature
        """
                
        # Make feature set
        print('Extracting features:')
        feature_set = make_unigram_feature_set(
            training_pos + training_neg,  # we use a concatenation of positive and negative training instances
            min_freq=min_freq, 
            mark_negation=mark_negation)
        
        print(' %d features' % len(feature_set))
                
        # Estimate model: 1/2) count        
        print('MLE: counting')        
        counts = [defaultdict(float), defaultdict(float)]
        for docs, y in [(training_pos, 1), (training_neg, 0)]:
            for doc in docs:  # for each document
                # we extract features
                fmap = make_feature_map(doc, 
                                        feature_set, 
                                        binary=binary, 
                                        mark_negation=mark_negation)
                # and gather counts for the pair (y, f)
                for f, n in fmap.items():
                    counts[y][f] += n  
                                
        # 2/2) Laplace-1 MLE
        #  we put EMPTY() is in the support
        print('MLE: smoothing')
        counts[0]['EMPTY()'] += 0
        counts[1]['EMPTY()'] += 0
        # and compute cpds using Laplace smoothing
        self._cpds = [
            make_cpd(counts[0], alpha, len(feature_set) + 1),  # we add 1 because we want EMPTY() to add towards total
            make_cpd(counts[1], alpha, len(feature_set) + 1)]
        print('MLE: done')
            
        # Store data
        self._binary = binary
        self._mark_negation = mark_negation
        self._alpha = alpha
        self._feature_set = feature_set
            
    def get_log_parameter(self, f, y):
        """Returns log P(f|y)"""
        return np.log(self._cpds[y].get(f, self._cpds[y]['EMPTY()']))
        
    def classify(self, doc):
        """
        This function classifies a document by extracting features <f_1...f_n> for it 
         and then computing 
            log P(<f_1...f_n>|Y=0) and log P(<f_1...f_n>|Y=1)
         and finally picking the best (that is, either Y=0 or Y=1).
        
        :param doc: a list of tokens
        :returns: 0 or 1 (the argmax of log P(<f_1...f_n>|y))
        """
        
        # initial probabilities
        probabilities = [0, 0]
        
        # create a feature map of the document
        fmap = make_feature_map(doc, feature_set=self._feature_set, 
                                binary=self._binary, mark_negation=self._mark_negation)
            
        # loop over the words in the feature map
        for key in fmap: 
            
            # get the log prob of the negative sentiment
            probabilities[0] += self.get_log_parameter(key, 0)

            #  get the log prob of the positive sentiment
            probabilities[1] += self.get_log_parameter(key, 1)
            
        # return the index of the highest probability (or 0 if they are equal)
        return np.argmax(probabilities)

This is how you should use the classifier:

```python
>>> classifier1 = NaiveBayesClassifier(
    training_pos, training_neg, 
    binary=True, mark_negation=False,
    alpha=1., min_freq=2)
```
```
Extracting features:
 8577 features
MLE: counting
MLE: smoothing
MLE: done
```

In [16]:
classifier1 = NaiveBayesClassifier(
    training_pos, training_neg, 
    binary=True, mark_negation=False,
    alpha=1., min_freq=2)

Extracting features:
 8577 features
MLE: counting
MLE: smoothing
MLE: done


## <a name="eval"> Evaluation

We evaluate binary classifiers on precision, recall, F1 and accuracy. See [Figure 4.4](https://web.stanford.edu/~jurafsky/slp3/4.pdf) and complete the code below:

<a name="ex5-4" style="color:red">**Exercise 5-4**</a> **[2 points]** Classify all documents in a dev set and compute the quantities in [Figure 4.4](https://web.stanford.edu/~jurafsky/slp3/4.pdf).

In [17]:
def evaluate_model(classifier, pos_docs, neg_docs):
    """
    :param classifier: an NaiveBayesClassifier object
    :param pos_docs: positive documents
    :param neg_docs: negative documents
    :returns: a dictionary containing the number of
        * true positives
        * true negatives
        * false positives
        * false negatives
     as well as 
        * accuracy
        * precision
        * recall 
        * and [F1](https://en.wikipedia.org/wiki/F1_score)
    """
    
    # initial values for TP, TN, FP, FN
    true_positives = 0
    true_negatives = 0
    false_positives = 0
    false_negatives = 0
    
    # loop over the positive documents
    for doc in pos_docs:
        
        # classify the document
        classification = classifier.classify(doc)
        
        # if the classification is positive mark it as TP
        if classification == 1:
            true_positives += 1
        
        # otherwise it is a FN
        else:
            false_negatives += 1
            
    # loop over the negative docs
    for doc in neg_docs:
        
        # classify the document
        classification = classifier.classify(doc)
        
        # if the classification is negative, mark it as TN
        if classification == 0:
            true_negatives += 1
            
        # otherwise it is a FP
        else:
            false_positives += 1
            
    # accuracy is ( TP + TN ) / ( TP + TN + FP + FN )
    accuracy = float(true_positives + true_negatives) / (true_positives + true_negatives + 
                                                    false_positives + false_negatives)
    
    # precision is TP / ( TP + FP )
    precision = float(true_positives) / (true_positives + false_positives)
    
    # recall is TP / ( TP + FN )
    recall = float(true_positives) / (true_positives + false_negatives)
    
    # F1-score is 2 * P * R  / ( P + R )
    f1_score = 2 * ((precision * recall) / (precision + recall))
    
    # create the result dictionary
    result = dict()
    
    # fill in the dict
    result['TP'] = true_positives
    result['TN'] = true_negatives
    result['FP'] = false_positives
    result['FN'] = false_negatives
    result['A'] = accuracy
    result['P'] = precision
    result['R'] = recall
    result['F1'] = f1_score
    
    #  return the dict
    return result

For example, our implementation yields:

```python
>>> classifier1 = NaiveBayesClassifier(
    training_pos, training_neg, 
    binary=True, mark_negation=False,
    alpha=1., min_freq=2)
```
```
Extracting features:
 8577 features
MLE: counting
MLE: smoothing
MLE: done
```
```python
>>> dev_metrics1 = evaluate_model(classifier1, dev_pos, dev_neg)
>>> print('Development')
>>> print('TP %d TN %d FP %d FN %d' % (dev_metrics1['TP'], dev_metrics1['TN'], dev_metrics1['FP'], dev_metrics1['FN']))
>>> print('P %.4f R %.4f A %.4f F1 %.4f' % (dev_metrics1['P'], dev_metrics1['R'], dev_metrics1['A'], dev_metrics1['F1']))
```
```
Development
TP 239 TN 268 FP 63 FN 92
P 0.7914 R 0.7221 A 0.7659 F1 0.7551
```
```python
>>> classifier2 = NaiveBayesClassifier(
    training_pos, training_neg, 
    binary=True, mark_negation=True,
    alpha=1., min_freq=2)
```
```
Extracting features:
 9581 features
MLE: counting
MLE: smoothing
MLE: done  
```
```python
>>> dev_metrics2 = evaluate_model(classifier2, dev_pos, dev_neg)
>>> print('Development')
>>> print('TP %d TN %d FP %d FN %d' % (dev_metrics2['TP'], dev_metrics2['TN'], dev_metrics2['FP'], dev_metrics2['FN']))
>>> print('P %.4f R %.4f A %.4f F1 %.4f' % (dev_metrics2['P'], dev_metrics2['R'], dev_metrics2['A'], dev_metrics2['F1']))
```
```
Development
TP 248 TN 273 FP 58 FN 83
P 0.8105 R 0.7492 A 0.7870 F1 0.7786
```

In [18]:
classifier1 = NaiveBayesClassifier(
    training_pos, training_neg, 
    binary=True, mark_negation=False,
    alpha=1., min_freq=2)

Extracting features:
 8577 features
MLE: counting
MLE: smoothing
MLE: done


In [19]:
dev_metrics1 = evaluate_model(classifier1, dev_pos, dev_neg)
print('Development')
print('TP %d TN %d FP %d FN %d' % (dev_metrics1['TP'], dev_metrics1['TN'], dev_metrics1['FP'], dev_metrics1['FN']))
print('P %.4f R %.4f A %.4f F1 %.4f' % (dev_metrics1['P'], dev_metrics1['R'], dev_metrics1['A'], dev_metrics1['F1']))

Development
TP 239 TN 268 FP 63 FN 92
P 0.7914 R 0.7221 A 0.7659 F1 0.7551


In [21]:
classifier2 = NaiveBayesClassifier(
    training_pos, training_neg, 
    binary=True, mark_negation=True,
    alpha=1., min_freq=2)

Extracting features:
 9581 features
MLE: counting
MLE: smoothing
MLE: done


In [22]:
dev_metrics2 = evaluate_model(classifier2, dev_pos, dev_neg)
print('Development')
print('TP %d TN %d FP %d FN %d' % (dev_metrics2['TP'], dev_metrics2['TN'], dev_metrics2['FP'], dev_metrics2['FN']))
print('P %.4f R %.4f A %.4f F1 %.4f' % (dev_metrics2['P'], dev_metrics2['R'], dev_metrics2['A'], dev_metrics2['F1']))

Development
TP 248 TN 273 FP 58 FN 83
P 0.8105 R 0.7492 A 0.7870 F1 0.7786


<a name="ex5-5" style="color:red">**Exercise 5-5**</a> **[4 points]** Use the dev set to choose the best configuration of 

* alpha (try values like 0.1, 0.5, 1.)
* and binary vs count

for a model that marks negation and one that does not. 

Then report performance on test set for your best model in each case. 

Points:
* 3 points for the search on dev set
* 1 point for the table of test results

In [23]:
def hyperparameters(alpha, binary, mark_negation, min_freq):
    """
    Get the best parameters for the NB classifier
    :param alpha: the pseudo count for smoothing 
    :param binary: binary feature map or not
    :param mark_negation: mark the negations or not
    :param min_freq: minimum frequency for words to include them
    :returns: a list of dictionaries with the results and
    
    NOTE: the parameters are lists with options that are evaluated
    """
    # result list
    result = []
    
    # loop over the alpha's
    for a in alpha:
        
        # try the binary options
        for b in binary:
            
            # try the mark negation options
            for mn in mark_negation:
                
                # loop over the min frequencies
                for mf in min_freq:
                    
                    # create the classifier
                    classifier = NaiveBayesClassifier(
                    training_pos, training_neg, 
                    binary=b, mark_negation=mn,
                    alpha=a, min_freq=mf)
                    
                    # evaluate the model
                    dev_metrics = evaluate_model(classifier, dev_pos, dev_neg)
                    
                    # add the current parameters to the results
                    dev_metrics['Alpha'] = a
                    dev_metrics['Binary'] = b
                    dev_metrics['Mark Negation'] = mn
                    dev_metrics['Min Frequency'] = mf
                    
                    # add the current result to the list
                    result.append(dev_metrics)
    
    # return the list
    return np.asarray(result)

In [24]:
# alpha values of 0, 0.1, 0.2, ..., 1.0
alpha = np.linspace(0, 1, 11)

# both options for the booleans
binary = [True, False]
mark_negation = [True, False]

# min frequency from 1 to 5
min_freq = range(1, 6)

# get all the results from the given parameter options
results = hyperparameters(alpha, binary, mark_negation, min_freq)

Extracting features:
 21805 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:


/usr/lib/python3/dist-packages/ipykernel_launcher.py:85: RuntimeWarning: divide by zero encountered in log


 9581 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 6283 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 4701 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 3739 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 18283 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 8577 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 5829 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 4448 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 3525 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 21805 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 9581 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 6283 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 4701 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 3739 features
MLE: cou

 21805 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 9581 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 6283 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 4701 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 3739 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 18283 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 8577 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 5829 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 4448 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 3525 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 21805 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 9581 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 6283 features
MLE: counting
MLE: smoothing
MLE: done
Extracting features:
 4701 features
MLE: co

MLE: smoothing
MLE: done


In [25]:
# get the best parameters for a certain score
def get_best_parameters(data, score):
    
    # get the index of the best parameters
    index = np.argmax([x[score] for x in data])
    
    # return the results with the best parameters
    return data[index]

In [26]:
# the best f1 score
best_F1 = get_best_parameters(results, 'F1')
print(best_F1)

{'TP': 258, 'TN': 262, 'FP': 69, 'FN': 73, 'A': 0.7854984894259819, 'P': 0.7889908256880734, 'R': 0.7794561933534743, 'F1': 0.7841945288753798, 'Alpha': 0.30000000000000004, 'Binary': False, 'Mark Negation': True, 'Min Frequency': 3}


In [27]:
# the best accuracy
best_A = get_best_parameters(results, 'A')
print(best_A)

{'TP': 250, 'TN': 273, 'FP': 58, 'FN': 81, 'A': 0.7900302114803626, 'P': 0.8116883116883117, 'R': 0.7552870090634441, 'F1': 0.7824726134585289, 'Alpha': 0.5, 'Binary': True, 'Mark Negation': True, 'Min Frequency': 2}


In [28]:
# the best precision
best_P = get_best_parameters(results, 'P')
print(best_P)

{'TP': 247, 'TN': 274, 'FP': 57, 'FN': 84, 'A': 0.7870090634441088, 'P': 0.8125, 'R': 0.7462235649546828, 'F1': 0.7779527559055119, 'Alpha': 0.5, 'Binary': False, 'Mark Negation': True, 'Min Frequency': 2}


In [29]:
# the best recall
best_R = get_best_parameters(results, 'R')
print(best_R)

{'TP': 259, 'TN': 242, 'FP': 89, 'FN': 72, 'A': 0.756797583081571, 'P': 0.7442528735632183, 'R': 0.7824773413897281, 'F1': 0.7628865979381443, 'Alpha': 0.0, 'Binary': True, 'Mark Negation': True, 'Min Frequency': 3}


In [30]:
# dev set results, test is below

import pandas as pd

# initial list of values
values = []

# loop over the results
for row in results:
    
    # create a new list
    dictlist = []
    
    # loop over the values in the dictionary of one row
    for value in row.values():
        
        # add the value to the list
        dictlist.append(value)
    
    # add the converted dict (now a list) to the values list as a row
    values.append(dictlist)
    
# get the keys
keys = [x for x in results[0].keys()] 
    
# create the pandas dataframe (can be printed as a table)
output = pd.DataFrame(values, columns=keys)

# print the dataframe as table
output

,TP,TN,FP,FN,A,P,R,F1,Alpha,Binary,Mark Negation,Min Frequency
0,183,267,64,148,0.679758,0.740891,0.552870,0.633218,0.0,True,True,1
1,224,259,72,107,0.729607,0.756757,0.676737,0.714514,0.0,True,True,2
2,259,242,89,72,0.756798,0.744253,0.782477,0.762887,0.0,True,True,3
3,249,253,78,82,0.758308,0.761468,0.752266,0.756839,0.0,True,True,4
4,237,251,80,94,0.737160,0.747634,0.716012,0.731481,0.0,True,True,5
5,177,258,73,154,0.657100,0.708000,0.534743,0.609294,0.0,True,False,1
6,220,257,74,111,0.720544,0.748299,0.664653,0.704000,0.0,True,False,2
7,251,243,88,80,0.746224,0.740413,0.758308,0.749254,0.0,True,False,3
8,243,253,78,88,0.749245,0.757009,0.734139,0.745399,0.0,True,False,4
9,234,257,74,97,0.741692,0.759740,0.706949,0.732394,0.0,True,False,5


In [31]:
# We have chosen the F1-score since precision and recall are both important for a good model
# and the F1-score is an evaluation of both
best_classifier = NaiveBayesClassifier(
    test_pos, test_neg, 
    binary=best_F1['Binary'], mark_negation=best_F1['Mark Negation'],
    alpha=best_F1['Alpha'], min_freq=best_F1['Min Frequency'])

Extracting features:
 1882 features
MLE: counting
MLE: smoothing
MLE: done


In [32]:
# get the test metrics
test_metrics = evaluate_model(best_classifier, dev_pos, dev_neg)

# print the table
print('Table of test results')
print('-------------------------')
for key, value in test_metrics.items():
    print(key, "\t|", value)

Table of test results
-------------------------
TP 	| 237
TN 	| 235
FP 	| 96
FN 	| 94
A 	| 0.7129909365558912
P 	| 0.7117117117117117
R 	| 0.716012084592145
F1 	| 0.713855421686747
